In [52]:
import tensorflow as tf
tf.__version__


'2.12.0'

In [53]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [54]:
subject = 'Amazonas - Consumo de Cimento (t)'
split_index = 6 #Referente aos 7 anos de input  

In [55]:
data = pd.read_csv('../Inputs Atuais/2014_01_model_input_AM.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data

,Ano,Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado,Amazonas - IDH Renda,Amazonas - IDH Educacao,Amazonas - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amazonas - Produção de Cimento (t),Amazonas - Longevidade,Amazonas - IDH,Amazonas - Consumo de Cimento (t)
0,2014,5.487107e+07,2.472487e+06,12.359116,4.412824e+07,0.666397,0.701630,12.610217,5.676413e+08,2.779340e+09,...,1004.431965,1277.324904,28.055466,22.412378,73.347472,1.524622,629.906817,0.761764,0.708325,771.567857
1,2015,5.527657e+07,2.446996e+06,12.336147,4.411441e+07,0.659448,0.714202,13.139356,5.179392e+08,2.561731e+09,...,1004.517645,1275.202120,28.150076,22.434009,72.952065,1.509930,585.627424,0.771049,0.713064,746.867486
2,2016,5.561997e+07,2.436680e+06,12.339032,4.416689e+07,0.658216,0.716766,13.263561,4.524144e+08,2.516859e+09,...,1004.590070,1250.291696,27.936211,22.456139,73.785027,1.450843,527.123225,0.770508,0.713252,739.457657
3,2017,5.581151e+07,2.432227e+06,12.358279,4.424802e+07,0.658599,0.717836,12.821072,4.213689e+08,2.497710e+09,...,1004.691076,1227.047962,27.873459,22.397053,73.760599,1.416718,447.665440,0.769554,0.713457,687.211000
4,2018,5.585158e+07,2.441182e+06,12.381581,4.430790e+07,0.657381,0.716548,12.662274,4.166580e+08,2.797145e+09,...,1004.787155,1200.261326,27.864216,22.349568,73.584375,1.381646,427.762220,0.768150,0.712134,350.177000
5,2019,5.559817e+07,2.458408e+06,12.387236,4.429218e+07,0.657597,0.716507,12.576164,4.462625e+08,3.484252e+09,...,1004.762314,1153.201551,27.753355,22.383859,74.220484,1.341209,464.893117,0.766254,0.711601,533.444000
6,2020,5.499113e+07,2.476121e+06,12.382493,4.418587e+07,0.657561,0.714826,12.440277,5.489412e+08,4.114911e+09,...,1004.721967,1102.300417,27.565392,22.396203,75.065647,1.309276,474.980247,0.763914,0.710292,595.320000
7,2021,5.402092e+07,2.486959e+06,12.396810,4.405505e+07,0.657611,0.711810,12.255143,6.638811e+08,4.421848e+09,...,1004.652726,1090.057669,27.575623,22.299417,74.609970,1.304082,468.326664,0.762178,0.708773,610.155000
8,2022,5.284405e+07,2.500011e+06,12.412850,4.388669e+07,0.659122,0.711066,12.276323,7.010332e+08,4.533216e+09,...,1004.593122,1096.634990,27.680371,22.221446,73.875828,1.313308,470.483830,0.763831,0.709571,640.274000


In [56]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado,Amazonas - IDH Renda,Amazonas - IDH Educacao,Amazonas - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amazonas - Produção de Cimento (t),Amazonas - Longevidade,Amazonas - IDH
0,5.487107e+07,2.472487e+06,12.359116,4.412824e+07,0.666397,0.701630,12.610217,5.676413e+08,2.779340e+09,1.954361e+10,...,38.217511,1004.431965,1277.324904,28.055466,22.412378,73.347472,1.524622,629.906817,0.761764,0.708325
1,5.527657e+07,2.446996e+06,12.336147,4.411441e+07,0.659448,0.714202,13.139356,5.179392e+08,2.561731e+09,1.327596e+10,...,38.739113,1004.517645,1275.202120,28.150076,22.434009,72.952065,1.509930,585.627424,0.771049,0.713064
2,5.561997e+07,2.436680e+06,12.339032,4.416689e+07,0.658216,0.716766,13.263561,4.524144e+08,2.516859e+09,1.011370e+10,...,39.689203,1004.590070,1250.291696,27.936211,22.456139,73.785027,1.450843,527.123225,0.770508,0.713252
3,5.581151e+07,2.432227e+06,12.358279,4.424802e+07,0.658599,0.717836,12.821072,4.213689e+08,2.497710e+09,8.456309e+09,...,39.521434,1004.691076,1227.047962,27.873459,22.397053,73.760599,1.416718,447.665440,0.769554,0.713457
4,5.585158e+07,2.441182e+06,12.381581,4.430790e+07,0.657381,0.716548,12.662274,4.166580e+08,2.797145e+09,7.937760e+09,...,41.396833,1004.787155,1200.261326,27.864216,22.349568,73.584375,1.381646,427.762220,0.768150,0.712134
5,5.559817e+07,2.458408e+06,12.387236,4.429218e+07,0.657597,0.716507,12.576164,4.462625e+08,3.484252e+09,1.091282e+10,...,42.694126,1004.762314,1153.201551,27.753355,22.383859,74.220484,1.341209,464.893117,0.766254,0.711601
6,5.499113e+07,2.476121e+06,12.382493,4.418587e+07,0.657561,0.714826,12.440277,5.489412e+08,4.114911e+09,1.625333e+10,...,41.168367,1004.721967,1102.300417,27.565392,22.396203,75.065647,1.309276,474.980247,0.763914,0.710292


In [57]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0    746.867486
1    739.457657
2    687.211000
3    350.177000
4    533.444000
5    595.320000
6    610.155000
7    640.274000
8           NaN
Name: Amazonas - Consumo de Cimento (t), dtype: float64

In [58]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado,Amazonas - IDH Renda,Amazonas - IDH Educacao,Amazonas - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amazonas - Produção de Cimento (t),Amazonas - Longevidade,Amazonas - IDH
0,-1.868792,1.782709,-0.058017,-1.058374,2.182699,-2.193749,-0.885700,1.779974,0.019134,2.003355,...,-1.182854,-1.539115,1.069051,0.892725,0.200018,-0.663153,1.327701,1.565524,-1.936041,-2.077111
1,-0.673033,-0.072827,-1.252234,-1.238278,-0.050952,0.051327,1.106694,0.870373,-0.621387,0.401153,...,-0.844881,-0.873340,1.020529,1.616656,0.829167,-1.668328,1.103821,0.968331,1.003254,0.621888
2,0.339576,-0.823758,-1.102216,-0.555608,-0.446931,0.509148,1.574370,-0.328802,-0.753466,-0.407217,...,-0.229267,-0.310562,0.451136,-0.019792,1.472838,0.449170,0.203409,0.179289,0.831972,0.728757
3,0.904397,-1.147886,-0.101545,0.499745,-0.323752,0.700156,-0.091755,-0.896969,-0.809829,-0.830898,...,-0.337974,0.474298,-0.080161,-0.499949,-0.245710,0.387069,-0.316604,-0.892353,0.529998,0.845342
4,1.022548,-0.496081,1.109992,1.278497,-0.715313,0.470225,-0.689687,-0.983185,0.071542,-0.963455,...,0.877196,1.220871,-0.692440,-0.570678,-1.626842,-0.060915,-0.851059,-1.160786,0.085450,0.092398
5,0.275303,0.757844,1.404019,1.074017,-0.645751,0.462893,-1.013922,-0.441390,2.094006,-0.202939,...,1.717780,1.027847,-1.768115,-1.418962,-0.629471,1.556156,-1.467268,-0.660004,-0.514633,-0.211274


In [59]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0    746.867486
1    739.457657
2    687.211000
3    350.177000
4    533.444000
5    595.320000
Name: Amazonas - Consumo de Cimento (t), dtype: float64

In [60]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado,Amazonas - IDH Renda,Amazonas - IDH Educacao,Amazonas - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amazonas - Produção de Cimento (t),Amazonas - Longevidade,Amazonas - IDH
6,-1.217033,1.49901,0.981704,-0.283871,-0.590777,0.150341,-1.223577,1.16956,2.035192,0.985175,...,0.650808,0.638482,-1.818433,-1.797281,-0.249089,1.993344,-1.455152,-0.475851,-1.050204,-0.832972


In [61]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

6    610.155
Name: Amazonas - Consumo de Cimento (t), dtype: float64

In [62]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=500, verbose=1)
    # Método estocástico e learning rate=0.01
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dense(2048, activation='tanh'),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(512, activation='tanh'),
        tf.keras.layers.Dense(64, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=5000, 
                      batch_size=len(train_input), 
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [63]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(10)]
    print(random_seeds)

    best_val_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, 0, seed)

        current_val_loss = min(history.history['val_loss'])
        print(f"val_loss: {current_val_loss}")

        if current_val_loss < best_val_loss:
            best_val_loss = current_val_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [64]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[1575691465, 2178513901, 2525264062, 3421911955, 1060873795, 1473528251, 1641351995, 3065356723, 335810618, 2893471653]


Step: 0 ___________________________________________
Epoch 1140: early stopping
val_loss: 10473.7060546875
winner_seed: 1575691465


Step: 1 ___________________________________________
Epoch 1355: early stopping
val_loss: 10473.7060546875


Step: 2 ___________________________________________
Epoch 1504: early stopping
val_loss: 957.2913208007812
winner_seed: 2525264062


Step: 3 ___________________________________________
Epoch 1299: early stopping
val_loss: 10473.7060546875


Step: 4 ___________________________________________
Epoch 1300: early stopping
val_loss: 957.2913208007812


Step: 5 ___________________________________________
Epoch 1530: early stopping
val_loss: 957.298828125


Step: 6 ___________________________________________
Epoch 1239: early stopping
val_loss: 2892.61328125


Step: 7 ___________________________________________
Epoch 1664: early stopping

In [65]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/5000
1/1 [==============================] - 0s 312ms/step - loss: 389712.7188 - val_loss: 344780.9062
Epoch 2/5000
1/1 [==============================] - 0s 28ms/step - loss: 366667.0938 - val_loss: 482162.7500
Epoch 3/5000
1/1 [==============================] - 0s 27ms/step - loss: 310812.9688 - val_loss: 383984.8438
Epoch 4/5000
1/1 [==============================] - 0s 26ms/step - loss: 273727.2812 - val_loss: 613493.0000
Epoch 5/5000
1/1 [==============================] - 0s 25ms/step - loss: 276950.7812 - val_loss: 501938.4375
Epoch 6/5000
1/1 [==============================] - 0s 28ms/step - loss: 281188.2812 - val_loss: 537595.2500
Epoch 7/5000
1/1 [==============================] - 0s 32ms/step - loss: 271418.2812 - val_loss: 515863.5938
Epoch 8/5000
1/1 [==============================] - 0s 32ms/step - loss: 262903.1875 - val_loss: 500287.8750
Epoch 9/5000
1/1 [==============================] - 0s 29ms/step - loss: 254682.2031 - val_loss: 484807.8438
Epoch 10/5000
1/1 

1/1 [==============================] - 0s 29ms/step - loss: 30884.8926 - val_loss: 67774.2109
Epoch 77/5000
1/1 [==============================] - 0s 25ms/step - loss: 29941.3379 - val_loss: 65895.5703
Epoch 78/5000
1/1 [==============================] - 0s 28ms/step - loss: 29027.3379 - val_loss: 64074.7383
Epoch 79/5000
1/1 [==============================] - 0s 28ms/step - loss: 28141.9375 - val_loss: 62309.4570
Epoch 80/5000
1/1 [==============================] - 0s 25ms/step - loss: 27284.2129 - val_loss: 60597.5117
Epoch 81/5000
1/1 [==============================] - 0s 25ms/step - loss: 26453.2988 - val_loss: 58937.0312
Epoch 82/5000
1/1 [==============================] - 0s 25ms/step - loss: 25648.3359 - val_loss: 57326.1211
Epoch 83/5000
1/1 [==============================] - 0s 24ms/step - loss: 24868.5293 - val_loss: 55763.1602
Epoch 84/5000
1/1 [==============================] - 0s 25ms/step - loss: 24113.0625 - val_loss: 54246.4023
Epoch 85/5000
1/1 [=======================

Epoch 152/5000
1/1 [==============================] - 0s 26ms/step - loss: 3383.3574 - val_loss: 9925.4990
Epoch 153/5000
1/1 [==============================] - 0s 26ms/step - loss: 3298.6904 - val_loss: 9711.3242
Epoch 154/5000
1/1 [==============================] - 0s 26ms/step - loss: 3216.6682 - val_loss: 9502.7988
Epoch 155/5000
1/1 [==============================] - 0s 27ms/step - loss: 3137.2024 - val_loss: 9299.7559
Epoch 156/5000
1/1 [==============================] - 0s 27ms/step - loss: 3060.2219 - val_loss: 9102.0615
Epoch 157/5000
1/1 [==============================] - 0s 25ms/step - loss: 2985.6438 - val_loss: 8909.5498
Epoch 158/5000
1/1 [==============================] - 0s 26ms/step - loss: 2913.3899 - val_loss: 8722.0713
Epoch 159/5000
1/1 [==============================] - 0s 28ms/step - loss: 2843.3948 - val_loss: 8539.4795
Epoch 160/5000
1/1 [==============================] - 0s 26ms/step - loss: 2775.5857 - val_loss: 8361.6592
Epoch 161/5000
1/1 [=================

1/1 [==============================] - 0s 25ms/step - loss: 914.8840 - val_loss: 2653.9854
Epoch 229/5000
1/1 [==============================] - 0s 25ms/step - loss: 907.2839 - val_loss: 2620.7351
Epoch 230/5000
1/1 [==============================] - 0s 24ms/step - loss: 899.9214 - val_loss: 2588.2036
Epoch 231/5000
1/1 [==============================] - 0s 29ms/step - loss: 892.7896 - val_loss: 2556.3875
Epoch 232/5000
1/1 [==============================] - 0s 30ms/step - loss: 885.8796 - val_loss: 2525.2649
Epoch 233/5000
1/1 [==============================] - 0s 28ms/step - loss: 879.1863 - val_loss: 2494.8147
Epoch 234/5000
1/1 [==============================] - 0s 25ms/step - loss: 872.7014 - val_loss: 2465.0278
Epoch 235/5000
1/1 [==============================] - 0s 29ms/step - loss: 866.4185 - val_loss: 2435.8779
Epoch 236/5000
1/1 [==============================] - 0s 32ms/step - loss: 860.3328 - val_loss: 2407.3564
Epoch 237/5000
1/1 [==============================] - 0s 25ms

1/1 [==============================] - 0s 26ms/step - loss: 692.6384 - val_loss: 1369.4833
Epoch 306/5000
1/1 [==============================] - 0s 26ms/step - loss: 691.9775 - val_loss: 1362.4272
Epoch 307/5000
1/1 [==============================] - 0s 33ms/step - loss: 691.3375 - val_loss: 1355.4972
Epoch 308/5000
1/1 [==============================] - 0s 28ms/step - loss: 690.7171 - val_loss: 1348.6925
Epoch 309/5000
1/1 [==============================] - 0s 29ms/step - loss: 690.1160 - val_loss: 1342.0121
Epoch 310/5000
1/1 [==============================] - 0s 28ms/step - loss: 689.5338 - val_loss: 1335.4510
Epoch 311/5000
1/1 [==============================] - 0s 28ms/step - loss: 688.9699 - val_loss: 1329.0128
Epoch 312/5000
1/1 [==============================] - 0s 26ms/step - loss: 688.4235 - val_loss: 1322.6921
Epoch 313/5000
1/1 [==============================] - 0s 24ms/step - loss: 687.8942 - val_loss: 1316.4841
Epoch 314/5000
1/1 [==============================] - 0s 30ms

...

1/1 [==============================] - 0s 34ms/step - loss: 671.4712 - val_loss: 957.2951
Epoch 1160/5000
1/1 [==============================] - 0s 36ms/step - loss: 671.4711 - val_loss: 957.2951
Epoch 1161/5000
1/1 [==============================] - 0s 35ms/step - loss: 671.4711 - val_loss: 957.2951
Epoch 1162/5000
1/1 [==============================] - 0s 35ms/step - loss: 671.4711 - val_loss: 957.2951
Epoch 1163/5000
1/1 [==============================] - 0s 34ms/step - loss: 671.4712 - val_loss: 957.2951
Epoch 1164/5000
1/1 [==============================] - 0s 34ms/step - loss: 671.4712 - val_loss: 957.2951
Epoch 1165/5000
1/1 [==============================] - 0s 33ms/step - loss: 671.4712 - val_loss: 957.2951
Epoch 1166/5000
1/1 [==============================] - 0s 35ms/step - loss: 671.4712 - val_loss: 957.2951
Epoch 1167/5000
1/1 [==============================] - 0s 37ms/step - loss: 671.4712 - val_loss: 957.2988
Epoch 1168/5000
1/1 [==============================] - 0s 41ms

1/1 [==============================] - 0s 34ms/step - loss: 671.4712 - val_loss: 957.2913
Epoch 1237/5000
1/1 [==============================] - 0s 33ms/step - loss: 671.4711 - val_loss: 957.2951
Epoch 1238/5000
1/1 [==============================] - 0s 35ms/step - loss: 671.4712 - val_loss: 957.2988
Epoch 1239/5000
1/1 [==============================] - 0s 34ms/step - loss: 671.4711 - val_loss: 957.2913
Epoch 1240/5000
1/1 [==============================] - 0s 35ms/step - loss: 671.4712 - val_loss: 957.2951
Epoch 1241/5000
1/1 [==============================] - 0s 33ms/step - loss: 671.4711 - val_loss: 957.2988
Epoch 1242/5000
1/1 [==============================] - 0s 35ms/step - loss: 671.4711 - val_loss: 957.2913
Epoch 1243/5000
1/1 [==============================] - 0s 35ms/step - loss: 671.4712 - val_loss: 957.2951
Epoch 1244/5000
1/1 [==============================] - 0s 34ms/step - loss: 671.4712 - val_loss: 957.2988
Epoch 1245/5000
1/1 [==============================] - 0s 33ms

1/1 [==============================] - 0s 34ms/step - loss: 671.4711 - val_loss: 957.2951
Epoch 1314/5000
1/1 [==============================] - 0s 35ms/step - loss: 671.4712 - val_loss: 957.2988
Epoch 1315/5000
1/1 [==============================] - 0s 36ms/step - loss: 671.4712 - val_loss: 957.2988
Epoch 1316/5000
1/1 [==============================] - 0s 32ms/step - loss: 671.4712 - val_loss: 957.2951
Epoch 1317/5000
1/1 [==============================] - 0s 34ms/step - loss: 671.4712 - val_loss: 957.2951
Epoch 1318/5000
1/1 [==============================] - 0s 35ms/step - loss: 671.4711 - val_loss: 957.2913
Epoch 1319/5000
1/1 [==============================] - 0s 36ms/step - loss: 671.4711 - val_loss: 957.2951
Epoch 1320/5000
1/1 [==============================] - 0s 38ms/step - loss: 671.4711 - val_loss: 957.2988
Epoch 1321/5000
1/1 [==============================] - 0s 37ms/step - loss: 671.4711 - val_loss: 957.2988
Epoch 1322/5000
1/1 [==============================] - 0s 33ms

1/1 [==============================] - 0s 35ms/step - loss: 671.4711 - val_loss: 957.2951
Epoch 1391/5000
1/1 [==============================] - 0s 34ms/step - loss: 671.4712 - val_loss: 957.2988
Epoch 1392/5000
1/1 [==============================] - 0s 34ms/step - loss: 671.4712 - val_loss: 957.2988
Epoch 1393/5000
1/1 [==============================] - 0s 34ms/step - loss: 671.4711 - val_loss: 957.2951
Epoch 1394/5000
1/1 [==============================] - 0s 35ms/step - loss: 671.4712 - val_loss: 957.2951
Epoch 1395/5000
1/1 [==============================] - 0s 33ms/step - loss: 671.4711 - val_loss: 957.2988
Epoch 1396/5000
1/1 [==============================] - 0s 37ms/step - loss: 671.4711 - val_loss: 957.2951
Epoch 1397/5000
1/1 [==============================] - 0s 38ms/step - loss: 671.4711 - val_loss: 957.2951
Epoch 1398/5000
1/1 [==============================] - 0s 35ms/step - loss: 671.4712 - val_loss: 957.2988
Epoch 1399/5000
1/1 [==============================] - 0s 34ms

1/1 [==============================] - 0s 36ms/step - loss: 671.4712 - val_loss: 957.2951
Epoch 1468/5000
1/1 [==============================] - 0s 39ms/step - loss: 671.4712 - val_loss: 957.2951
Epoch 1469/5000
1/1 [==============================] - 0s 40ms/step - loss: 671.4712 - val_loss: 957.2988
Epoch 1470/5000
1/1 [==============================] - 0s 34ms/step - loss: 671.4711 - val_loss: 957.2988
Epoch 1471/5000
1/1 [==============================] - 0s 35ms/step - loss: 671.4711 - val_loss: 957.2951
Epoch 1472/5000
1/1 [==============================] - 0s 33ms/step - loss: 671.4711 - val_loss: 957.2988
Epoch 1473/5000
1/1 [==============================] - 0s 35ms/step - loss: 671.4712 - val_loss: 957.2988
Epoch 1474/5000
1/1 [==============================] - 0s 38ms/step - loss: 671.4712 - val_loss: 957.2951
Epoch 1475/5000
1/1 [==============================] - 0s 36ms/step - loss: 671.4712 - val_loss: 957.2988
Epoch 1476/5000
1/1 [==============================] - 0s 39ms

In [66]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 52ms/step
[610.155](test_target) - [[564.3798]](prediction) = 45.77517822265622


In [67]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [68]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado,Amazonas - IDH Renda,Amazonas - IDH Educacao,Amazonas - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amazonas - Produção de Cimento (t),Amazonas - Longevidade,Amazonas - IDH
0,-1.257162,1.357708,1.404781,-0.372516,1.400356,-1.396395,-1.391375,1.165029,1.396038,1.335266,...,-1.090581,-1.257499,0.791818,0.093887,-1.220061,-0.041298,0.924861,1.164534,-1.412307,-1.413415
1,0.067638,-0.336112,-0.843613,-0.995234,-0.529148,0.504391,0.476467,0.111769,-0.502296,-0.264151,...,-0.234437,0.068372,0.618866,1.175100,-0.009315,-1.203574,0.464108,0.112641,0.769731,0.665548
2,1.189524,-1.021596,-0.561168,1.367750,-0.871208,0.892004,0.914907,-1.276798,-0.893743,-1.071116,...,1.325018,1.189127,-1.410684,-1.268986,1.229376,1.244872,-1.388969,-1.277174,0.642576,0.747867


train_target:


,Amazonas - Consumo de Cimento (t)
0,746.867486
1,739.457657
2,687.211000


test_input:


...

,Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado,Amazonas - IDH Renda,Amazonas - IDH Educacao,Amazonas - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amazonas - Produção de Cimento (t),Amazonas - Longevidade,Amazonas - IDH
6,-1.217033,1.49901,0.981704,-0.283871,-0.590777,0.150341,-1.223577,1.16956,2.035192,0.985175,...,0.650808,0.638482,-1.818433,-1.797281,-0.249089,1.993344,-1.455152,-0.475851,-1.050204,-0.832972


test_target:


,Amazonas - Consumo de Cimento (t)
6,610.155


Epoch 1504: early stopping
1/1 [==============================] - 0s 52ms/step
Error: 45.77517822265622




[687.208740234375, 518.6871337890625, 441.80859375, 564.3798217773438]

In [69]:
display(targets)
display(predictions)

[350.177, 533.444, 595.32, 610.155]

[687.208740234375, 518.6871337890625, 441.80859375, 564.3798217773438]

In [70]:
mae = mean_absolute_error(predictions, targets)
mae

137.76879772949218

In [71]:
porcentage = mae/np.mean(targets)
porcentage

0.26378643725227024